# 2. Amazon Bedrock Knowledge Bases - GraphRAG 생성
- Amazon SageMaker AI - Notebook - JupyterLab 환경에서 테스트 되었습니다.
- Kernel : conda_python3

![graphrag00](../img/graphrag-00.png)
Amazon Bedrock의 Knowledge Bases와 Amazon Neptune Analytics의 통합으로 GraphRAG를 손쉽게 구현할 수 있습니다. GraphRAG는 기존 RAG 기술을 확장하여 문서 간 관계를 그래프로 구성함으로써 복잡한 문서에서 더 정확하고 맥락에 맞는 정보를 추출합니다. 이 기술은 문서 청크와 비정형 데이터에서 발견된 엔티티 및 관계를 자동으로 연결하는 그래프를 생성하여, 그래프 전문 지식 없이도 더 포괄적이고 정확한 응답을 제공합니다.

> 더 자세한 내용은 AWS 기술 블로그 [비정형 데이터! Amazon Bedrock으로 제대로 활용하기](https://aws.amazon.com/ko/blogs/tech/bedrock-data-automation-graphrag/) 블로그를 참조하세요.

## 1. Setup

### 필요한 라이브러리 설치

In [2]:
# 필요한 라이브러리 설치 (설치 후 커널 재부팅 필요)
!pip install -q boto3 --upgrade
!pip install -q awscli --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.239.3 requires numpy<2.0,>=1.9.0, but you have numpy 2.2.3 which is incompatible.


### 라이브러리 임포트 및 세션 설정

In [23]:
# 필요한 라이브러리 임포트
import boto3
import json
import time
import os
import uuid
from datetime import datetime
import pandas as pd
from tqdm.notebook import tqdm

# AWS 리전 설정
region = boto3.session.Session().region_name
print(f"현재 AWS 리전: {region}")

# 세션 및 클라이언트 설정
session = boto3.session.Session(region_name=region)
bedrock = session.client('bedrock')
bedrock_runtime = session.client('bedrock-runtime')
bedrock_agent = session.client('bedrock-agent')
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')
s3 = session.client('s3')


현재 AWS 리전: us-west-2


In [4]:
print(boto3.__version__)

1.37.3


## 2. KB - GraphRAG 생성 
- [참고] boto3 및 awscli를 통한 KB - GraphRAG 생성은 2025년 1분기에 지원 예정 입니다.

<b> Amazon Bedrock 콘솔 화면에서 Knowledge Bases 를 선택 합니다. </b>

![graphrag01](../img/graphrag-01.png)

<b> Create 버튼을 누르고 Knowledge Base with vector store를 선택합니다. </b>

![graphrag03](../img/graphrag-02.png)

<b> 다음과 같이 정보를 입력 및 선택 합니다. 
- Knowledge Base name : graphrag-workshop
- IAM permission : Create and use a new service role 선택
- Choose data source : Amazon S3 선택
그리고 Next 버튼을 누릅니다.
</b>

![graphrag03](../img/graphrag-03.png)

<b> 다음과 같이 정보를 입력 및 선택 합니다. 
- Data source name : 10-q
- S3 URI : <앞서 생성된 S3 경로 s3://~~~/data/> 를 입력 또는 선택 합니다.
그리고 Next 버튼을 누릅니다.
</b>

![graphrag04](../img/graphrag-04.png)

<b> 다음과 같이 정보를 입력 및 선택 합니다. 
- Embeddings model : Titan Text Embeddings V2
- Vector store : Amazon Neptune Analytics (GraphRAG) - Preview
그리고 Next 버튼을 누릅니다.
</b>

![graphrag05](../img/graphrag-05.png)

<b> KB 생성 정보를 확인 후, Create Knowledge Base 버튼을 누릅니다.
</b>

![graphrag06](../img/graphrag-06.png)

<b> 수 분후 KB - GraphRAG가 생성되며, 여기서 다음 정보를 확인합니다.
- Knowledge Base ID

해당 정보는 다음 단계 수행을 위해 변수에 값을 저장 합니다.
</b>

![graphrag07](../img/graphrag-07.png)

In [17]:
# 생성한 KB - GraphRAG에 대한 kb_id 값 입력
# 예시 : kb_id = "WSPOHGBATW"


kb_id = "본인 환경의 kb_id"


### KB 생성 완료 대기

<b> Data source 에서 Data Sync를 수행합니다.
- Data source (10-q) 를 선택 합니다.
- Sync 버튼을 누릅니다.
몇 분 기다리면 Data Sync 가 완료 됩니다. 완료 되면, Status : Available 상태로 표시 됩니다.
</b>

![graphrag09](../img/graphrag-09.png)

---

## 3. KB - GraphRAG 쿼리 실행
- Claude 3.5 Sonnet 사용

In [34]:
# kb_id 값 확인

print(kb_id)

WSPOHGBATW


In [37]:
def query_knowledge_base(query_text, model_id, max_tokens=1000):
    try:
        # 검색 요청 구성
        retrieve_response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={
                'text': query_text
            },
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 3,
                    'overrideSearchType': 'SEMANTIC'                    
                }
            }
        )
        
        # 검색 결과 확인
        retrieved_results = retrieve_response.get('retrievalResults', [])
        
        if not retrieved_results:
            print("검색 결과가 없습니다.")
            return None
        
        # 검색 결과를 컨텍스트로 사용
        context = ""
        for i, result in enumerate(retrieved_results):
            content = result['content']['text']
            source = result.get('location', {}).get('s3Location', {}).get('uri', '알 수 없는 소스')
            score = result.get('score', 0)
            
            context += f"\n\n참고 문서 {i+1} (관련성 점수: {score}):\n{content}\n"
            print(f"검색 결과 {i+1}: 관련성 점수 {score}")
        
        # Claude에 질의 구성
        prompt = f"""
사용자의 질문에 답변해 주세요. 다음 참고 문서의 정보를 활용하세요:

{context}

사용자 질문: {query_text}

답변:
"""
        
        # Claude에 질의 요청
        response = bedrock_runtime.converse(
            modelId=modelId,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "text": prompt
                        }
                    ]
                }
            ],
            inferenceConfig={
                "maxTokens": max_tokens,
                "temperature": 0.7,
                "topP": 0.9
            }
        )
        
        # 응답 추출 및 반환
        answer = response['output']['message']['content'][0]['text']
        return answer
        
    except Exception as e:
        print(f"쿼리 실행 중 오류 발생: {e}")
        return None


### 사용자 쿼리 테스트

In [40]:
# 쿼리 예제 실행
test_queries = [
    "Amazon의 총 순매출은 시간이 지남에 따라 어떻게 변화했나요?",
    "Amazon의 운영 비용 증가가 각 사업 부문의 수익성과 전반적인 재무 성과에 어떤 영향을 미쳤으며, 이는 다른 재무 지표들과 어떤 연관성을 보이나요?"
]

modelId="us.anthropic.claude-3-5-sonnet-20241022-v2:0"    # Claude 3.5 Sonnet v2 (cross-inference)
#modelId="us.anthropic.claude-3-7-sonnet-20250219-v1:0"    # Claude 3.7 Sonnet (cross-inference)

for query in test_queries:
    print(f"\n질문: {query}")
    answer = query_knowledge_base(query, modelId)
    if answer:
        print(f"\n답변:\n{answer}")
    print("-" * 80)



질문: Amazon의 총 순매출은 시간이 지남에 따라 어떻게 변화했나요?
검색 결과 1: 관련성 점수 1.045573
검색 결과 2: 관련성 점수 1.035476
검색 결과 3: 관련성 점수 1.02006

답변:
제공된 데이터를 바탕으로 Amazon의 총 순매출(Total net sales) 변화를 시간순으로 정리하면 다음과 같습니다:

2021년 3분기: 1,108억 달러
2022년 3분기: 1,271억 달러
2023년 3분기: 1,431억 달러

이를 분석해보면:
- 2021년에서 2022년 3분기 사이: 약 14.7% 증가
- 2022년에서 2023년 3분기 사이: 약 12.6% 증가

전반적으로 Amazon의 총 순매출은 꾸준한 성장세를 보이고 있습니다. 이는 제품 판매(Net product sales)와 서비스 판매(Net service sales) 모두의 증가에 기인합니다. 특히 서비스 부문의 성장이 두드러지는 것을 볼 수 있습니다.

2022년 4분기에 대해서는 1,400억 달러에서 1,480억 달러 사이의 매출을 예상한다고 가이던스를 제시했습니다.
--------------------------------------------------------------------------------

질문: Amazon의 운영 비용 증가가 각 사업 부문의 수익성과 전반적인 재무 성과에 어떤 영향을 미쳤으며, 이는 다른 재무 지표들과 어떤 연관성을 보이나요?
검색 결과 1: 관련성 점수 1.069845
검색 결과 2: 관련성 점수 0.9136758
검색 결과 3: 관련성 점수 0.7466082

답변:
제공된 데이터를 바탕으로 Amazon의 운영 비용 증가와 재무 성과의 관계를 다음과 같이 분석할 수 있습니다:

1. 매출 대비 비용 증가:
- 2022년 3분기 총 매출은 1,271억 달러로 전년 동기 1,108억 달러 대비 증가했으나
- 같은 기간 운영 비용은 1,246억 달러로 전년 동기 1,060억 달러에서 크게 증가

2. 주요 비용 항목별 증가

---

## 5. Clean-up

In [ ]:
# 워크샵에서 생성된 리소스 ID/이름 설정

kb_id = "여기에 Knowledge Base ID 입력"
bucket_name = "여기에 S3 버킷 이름 입력"


# AWS 클라이언트 초기화
import boto3
region = boto3.session.Session().region_name
bedrock_agent = boto3.client('bedrock-agent', region_name=region)
s3 = boto3.client('s3', region_name=region)
iam = boto3.client('iam', region_name=region)

print(f"리소스 정리를 위한 변수가 설정되었습니다: region: {region}")

In [ ]:
# Knowledge Base 삭제
try:
    print(f"Knowledge Base 삭제 중: {kb_id}")
    response = bedrock_agent.delete_knowledge_base(knowledgeBaseId=kb_id)
    print(f"Knowledge Base 삭제 요청이 성공적으로 제출되었습니다.")
    print("상태: 백그라운드에서 삭제 진행 중...")
except Exception as e:
    print(f"Knowledge Base 삭제 중 오류 발생: {e}")

In [ ]:
# S3 버킷의 모든 객체 삭제 후 버킷 삭제
import time

try:
    # 모든 객체 삭제
    print(f"S3 버킷 내 모든 객체 삭제 중: {bucket_name}")

    # 모든 객체 나열 및 삭제
    paginator = s3.get_paginator('list_objects_v2')
    object_count = 0

    for page in paginator.paginate(Bucket=bucket_name):
        if 'Contents' in page:
            objects = [{'Key': obj['Key']} for obj in page['Contents']]
            s3.delete_objects(Bucket=bucket_name, Delete={'Objects': objects})
            object_count += len(objects)

    print(f"{object_count}개 객체 삭제 완료")

    # 버전 관리된 객체 삭제 (필요시)
    try:
        paginator = s3.get_paginator('list_object_versions')
        version_count = 0

        for page in paginator.paginate(Bucket=bucket_name):
            delete_list = []

            # 버전 삭제
            if 'Versions' in page:
                delete_list.extend([{'Key': obj['Key'], 'VersionId': obj['VersionId']} for obj in page['Versions']])

            # 삭제 마커 제거
            if 'DeleteMarkers' in page:
                delete_list.extend([{'Key': obj['Key'], 'VersionId': obj['VersionId']} for obj in page['DeleteMarkers']])

            if delete_list:
                s3.delete_objects(Bucket=bucket_name, Delete={'Objects': delete_list})
                version_count += len(delete_list)

        if version_count > 0:
            print(f"{version_count}개 객체 버전/삭제 마커 제거 완료")
    except Exception as e:
        print(f"버전 제거 중 오류(무시 가능): {e}")

    # 잠시 대기 (모든 객체가 삭제되기를 기다림)
    time.sleep(3)

    # 버킷 삭제
    print(f"S3 버킷 삭제 중: {bucket_name}")
    s3.delete_bucket(Bucket=bucket_name)
    print(f"S3 버킷 삭제 완료: {bucket_name}")

except Exception as e:
    print(f"S3 버킷 삭제 중 오류 발생: {e}")